# Getting and cleaning data

For the project, we retrive the dataset using the api for the open data of the City of Toronto. We clean duplicate identical rows that represent the same event and offence. Then, we examine the yearly counts and filter out old years with insufficient data. Finally, we save the dataset as a csv to be used in the analysis.

In [1]:
# Dependencies
from pathlib import Path
import pandas as pd
import requests
from pprint import pprint

## Get data using API

The City of Toronto Open Data API has an upper limit for the number of records that we can get with each request. Therefore, we adapted an algorithm to retrieve the whole dataset using the limit and offset parameters (found in [this source](https://support.smartbear.com/qacomplete/docs/developer/api/rest/api/reference/paging.html)).

Based on the documentation for the API and dataset, we set the url for our requests.

We then get the dataset metadata to find the total number of records. We do this by setting the limit parameter to 0 so we only retrieve the metadata.

In [2]:
# Base url for all toronto open data
base_url = "https://ckan0.cf.opendata.inter.prod-toronto.ca"

# Datasets are called "packages". Each package can contain many "resources"
# To retrieve the metadata for this package and its resources, use the package name in this page's URL:
url = base_url + "/api/3/action/package_show"
p = {"id": "major-crime-indicators"}
# get package resources
package = requests.get(url, params = p).json()
# Look at resources and find the one that is datastore_active
for resource in package["result"]["resources"]:
    # once we find the datastore_active resource, get the id
    if resource["datastore_active"]:
        resource_id = resource["id"]
# using that id and limit 0, get just the metadata first to find total items
# based on this algorithm:
# https://support.smartbear.com/qacomplete/docs/developer/api/rest/api/reference/paging.html
limit = 0

# build url for retrieve data
url = base_url + "/api/3/action/datastore_search"
# include resource id and limit in url parameters
p = {
    'id': resource_id,
    'limit': limit
}
# get metadata, and pprint it to find total items
metadata = requests.get(url, params = p).json()
pprint(metadata)

{'help': 'https://ckan0.cf.opendata.inter.prod-toronto.ca/api/3/action/help_show?name=datastore_search',
 'result': {'_links': {'next': '/api/3/action/datastore_search?id=af452875-cfdd-4596-a08a-7b93b65ea4f0&limit=0&offset=0',
                       'start': '/api/3/action/datastore_search?id=af452875-cfdd-4596-a08a-7b93b65ea4f0&limit=0'},
            'fields': [{'id': '_id', 'type': 'int'},
                       {'id': 'EVENT_UNIQUE_ID',
                        'info': {'notes': 'Offence Number'},
                        'type': 'text'},
                       {'id': 'REPORT_DATE',
                        'info': {'notes': 'Date Offence was Reported'},
                        'type': 'date'},
                       {'id': 'OCC_DATE',
                        'info': {'notes': 'Date of Offence'},
                        'type': 'date'},
                       {'id': 'REPORT_YEAR',
                        'info': {'notes': 'Year Offence was Reported'},
                        'type': 'f

Looking at the metadata, we find the total number of records within the item named 'result' and then 'total', so we save it to a variable. We also note that the actual data will be contained within the same 'result' item but in the subitem 'records'.

In [3]:
# get total items from metadata
total_items = metadata['result']['total']
total_items

323296

We are now ready to make the requests to retrieve the data. We set the limit parameter to the maximum that can be retrieved each time, based on the documentation (i.e. 32000). The offset parameter, which sets the starting record to retrieve will start at 0, since we want to start with the first record and increase by the limit amount each time we make a request. Each request will extend a list that contains the data and add the records retrieved in that request. We will repeat the request until the offset parameter is above the total number we found in the metadata. Then, we create a pandas dataframe using the final data list. At the end, we compare the total number that we got initially to the number of rows in the dataframe, to make sure they are equal.

In [4]:
# Update parameters to include both limit and offset
# From documentation, we know the max limit is 32000
# so we set, limit to that
p['limit'] = 32000

# offset will start at 0 and increase by 32000 each loop time
p['offset'] = 0

# create list to contain data
data_list = []

n = 1

# While offset is under total_items
while p['offset'] < total_items:
    # log number of request and parameters to be used
    print(f"request #{n} with parameters: {p}")
    # get data that is inside 'result' and 'records'
    data = requests.get(url, params = p).json()['result']['records']
    # extend data list
    data_list.extend(data)
    # To finish loop section, increase offset by limit
    p['offset'] += p['limit']
    n += 1

# Create dataframe with data
df = pd.DataFrame(data_list)
# print total length of dataframe, to confirm we got everything
print(f"\nThe total items in dataset is {total_items} and we got {len(df)}.")

# Check head of dataframe
df.head()


request #1 with parameters: {'id': 'af452875-cfdd-4596-a08a-7b93b65ea4f0', 'limit': 32000, 'offset': 0}
request #2 with parameters: {'id': 'af452875-cfdd-4596-a08a-7b93b65ea4f0', 'limit': 32000, 'offset': 32000}
request #3 with parameters: {'id': 'af452875-cfdd-4596-a08a-7b93b65ea4f0', 'limit': 32000, 'offset': 64000}
request #4 with parameters: {'id': 'af452875-cfdd-4596-a08a-7b93b65ea4f0', 'limit': 32000, 'offset': 96000}
request #5 with parameters: {'id': 'af452875-cfdd-4596-a08a-7b93b65ea4f0', 'limit': 32000, 'offset': 128000}
request #6 with parameters: {'id': 'af452875-cfdd-4596-a08a-7b93b65ea4f0', 'limit': 32000, 'offset': 160000}
request #7 with parameters: {'id': 'af452875-cfdd-4596-a08a-7b93b65ea4f0', 'limit': 32000, 'offset': 192000}
request #8 with parameters: {'id': 'af452875-cfdd-4596-a08a-7b93b65ea4f0', 'limit': 32000, 'offset': 224000}
request #9 with parameters: {'id': 'af452875-cfdd-4596-a08a-7b93b65ea4f0', 'limit': 32000, 'offset': 256000}
request #10 with parameters

,_id,EVENT_UNIQUE_ID,REPORT_DATE,OCC_DATE,REPORT_YEAR,REPORT_MONTH,REPORT_DAY,REPORT_DOY,REPORT_DOW,REPORT_HOUR,...,LOCATION_TYPE,PREMISES_TYPE,UCR_CODE,UCR_EXT,OFFENCE,MCI_CATEGORY,HOOD_158,NEIGHBOURHOOD_158,HOOD_140,NEIGHBOURHOOD_140
0,1,GO-20141262074,2014-01-01,1998-06-01,2014,January,1,1,Wednesday,12.0,...,"Apartment (Rooming House, Condo)",Apartment,1480,110,Administering Noxious Thing,Assault,38,Lansing-Westgate,38,Lansing-Westgate (38)
1,2,GO-20141260701,2014-01-01,2014-01-01,2014,January,1,1,Wednesday,3.0,...,"Commercial Dwelling Unit (Hotel, Motel, B & B,...",Commercial,2120,200,B&E,Break and Enter,70,South Riverdale,70,South Riverdale (70)
2,3,GO-20141260889,2014-01-01,2014-01-01,2014,January,1,1,Wednesday,4.0,...,"Apartment (Rooming House, Condo)",Apartment,1430,100,Assault,Assault,74,North St.James Town,74,North St.James Town (74)
3,4,GO-20141260973,2014-01-01,2014-01-01,2014,January,1,1,Wednesday,4.0,...,"Streets, Roads, Highways (Bicycle Path, Privat...",Outside,2130,210,Theft Over,Theft Over,NSA,NSA,NSA,NSA
4,5,GO-20141261050,2014-01-01,2014-01-01,2014,January,1,1,Wednesday,4.0,...,"Streets, Roads, Highways (Bicycle Path, Privat...",Outside,1430,100,Assault,Assault,69,Blake-Jones,66,Danforth (66)


## Clean duplicates

The dataset page description says that: 

> This data is provided at the offence and/or victim level, therefore one occurrence number may have several rows of data associated to the various MCIs used to categorize the occurrence.

Therefore, we decided to consider rows that are identical (i.e. same event ID and Offence) as a single data point, keeping those with the same event ID but different Offence name as separate.

So now we drop duplicate row values that have the same event id and offence.


In [5]:
no_duplicates_df = df.drop_duplicates(['EVENT_UNIQUE_ID', 'OFFENCE'])

# confirm that there are no duplicates of the same event and offence
no_duplicates_df[['EVENT_UNIQUE_ID', 'OFFENCE']].value_counts(ascending=False)


EVENT_UNIQUE_ID  OFFENCE               
GO-20141035797   Assault                   1
GO-20201253785   B&E                       1
GO-2020125466    Assault Bodily Harm       1
GO-20201254652   Theft Of Motor Vehicle    1
GO-20201254386   Theft Of Motor Vehicle    1
                                          ..
GO-20171529777   Assault                   1
GO-20171529763   Theft Of Motor Vehicle    1
GO-20171529587   B&E                       1
GO-20171529586   Assault                   1
GO-2022999907    Assault                   1
Length: 299830, dtype: int64

## Check annual counts

We now check how many records we have per year, to decide if we are using all years or just some of them.

In [6]:
# Check counts of occurance year

df_by_year = no_duplicates_df.groupby('OCC_YEAR').size()

df_by_year

OCC_YEAR
2000.0       27
2001.0       20
2002.0       18
2003.0       13
2004.0       25
2005.0       26
2006.0       12
2007.0       31
2008.0       44
2009.0       70
2010.0       95
2011.0      129
2012.0      185
2013.0      566
2014.0    30202
2015.0    30641
2016.0    31157
2017.0    32790
2018.0    34772
2019.0    37114
2020.0    32456
2021.0    32114
2022.0    37226
dtype: int64

From these counts, we see that data that occured before 2014 is likely incomplete, so we decided to filter out the years before 2014.

In [7]:
# Keep only 2014 and over
over_2014_df = (no_duplicates_df
    .loc[no_duplicates_df['OCC_YEAR'] >= 2014]
)
# check result
over_2014_df.groupby('OCC_YEAR').size()

OCC_YEAR
2014.0    30202
2015.0    30641
2016.0    31157
2017.0    32790
2018.0    34772
2019.0    37114
2020.0    32456
2021.0    32114
2022.0    37226
dtype: int64

## Write out data

Finally, we write the clean dataframe out to a csv file, so we can use it for analysis.

In [8]:
# save data from 2014 and over
over_2014_df.to_csv(
    Path('resources', 'mci_no_duplicates_2014_and_over.csv'),
    index=False
)
